In [5]:
ReloadProject('deep_learning')

notebook_init.py imported and reloaded
forwarded symbol: A
forwarded symbol: Activation
forwarded symbol: Dense
forwarded symbol: InputLayer
forwarded symbol: Model
forwarded symbol: Sequential
forwarded symbol: T
reloaded: circular_world_env
forwarded symbol: circular_world_env
reloaded: environment_impl
forwarded symbol: environment_impl
reloaded: gym
forwarded symbol: gym
reloaded: keras
forwarded symbol: keras
reloaded: layers
forwarded symbol: layers
reloaded: logging
forwarded symbol: logging
reloaded: models
forwarded symbol: models
reloaded: numpy
forwarded symbol: numpy
reloaded: numpy_util
forwarded symbol: numpy_util
reloaded: optimizers
forwarded symbol: optimizers
reloaded: policy_impl
forwarded symbol: policy_impl
reloaded: q_base
forwarded symbol: q_base
reloaded: qfunc_impl
forwarded symbol: qfunc_impl
reloaded: runner_impl
forwarded symbol: runner_impl


# My models

## 20-20-20
Try the model that has shape (20, 20, 20).

In [3]:
batch_size = 64  # used in qfunc and runner.
env = environment_impl.GymEnvironment(gym.make('LunarLander-v2'))
qfunc = qfunc_impl.DQN(
    model=qfunc_impl.CreateModel(
        state_shape=env.GetStateShape(),
        action_space_size=env.GetActionSpaceSize(),
        hidden_layer_sizes=(20, 20, 20)),
    training_batch_size=batch_size,
    discount_factor=0.99,
)
runner = runner_impl.ExperienceReplayRunner(experience_capacity=100000, experience_sample_batch_size=batch_size)

In [4]:
%%time
# Train
logging.ENV.debug_verbosity = 3
policy = policy_impl.GreedyPolicyWithRandomness(epsilon=1.0)
runner.Run(env=env, qfunc=qfunc, policy=policy, num_of_episodes=500)

[22:33:35] q_base.py:415 Episode 0/500: episode_total_reward = -182.99, episode_steps=59
[22:34:51] q_base.py:417 Episode 100/500: episode_total_reward = -379.66, episode_steps=93
[22:37:40] q_base.py:417 Episode 200/500: episode_total_reward = -358.49, episode_steps=109
[22:42:00] q_base.py:417 Episode 300/500: episode_total_reward = -284.88, episode_steps=110
[22:47:40] q_base.py:417 Episode 400/500: episode_total_reward = -136.33, episode_steps=101
CPU times: user 20min 42s, sys: 14.9 s, total: 20min 56s
Wall time: 21min 23s


In [7]:
%%time
# Train
logging.ENV.debug_verbosity = 3
policy = policy_impl.GreedyPolicyWithRandomness(epsilon=0.3)
runner.Run(env=env, qfunc=qfunc, policy=policy, num_of_episodes=500)

[23:07:12] q_base.py:415 Episode 0/500: episode_total_reward = -146.07, episode_steps=367
[23:59:58] q_base.py:417 Episode 100/500: episode_total_reward = -60.58, episode_steps=117
[01:58:06] q_base.py:417 Episode 200/500: episode_total_reward = -67.60, episode_steps=639
[03:58:17] q_base.py:417 Episode 300/500: episode_total_reward = 76.50, episode_steps=999


KeyboardInterrupt: 

In [ ]:
%%time
# Train
logging.ENV.debug_verbosity = 3
policy = policy_impl.GreedyPolicyWithRandomness(epsilon=0.1)
runner.Run(env=env, qfunc=qfunc, policy=policy, num_of_episodes=500)

[04:36:03] q_base.py:415 Episode 0/500: episode_total_reward = -54.28, episode_steps=999


In [ ]:
%%time
# Test
logging.ENV.debug_verbosity = 4
policy = policy_impl.GreedyPolicy()
runner.Run(env=env, qfunc=qfunc, policy=policy, num_of_episodes=100)

In [16]:
# qfunc.SaveModel('saved_models/lunarlander_shape_20-20-20_rmsprop_gamma_099.model')

TypeError: super(type, obj): obj must be an instance or subtype of type

In [17]:
qfunc._model.save_weights('saved_models/lunarlander_shape_20-20-20_rmsprop_gamma_099.weights')